# Imports

In [ ]:
import numpy as np
import scipy.linalg as spla
import scipy.io as spio
import matplotlib.pyplot as plt
from matplotlib import colors

from pymor.models.iosys import LTIModel
from pymor.operators.constructions import LincombOperator
from pymor.operators.numpy import NumpyMatrixOperator
from pymor.parameters.functionals import ProjectionParameterFunctional

In [ ]:
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True

# Build model

In [ ]:
mat = spio.loadmat('../lectures/data/ABCE.mat')
mat.keys()

In [ ]:
A0 = NumpyMatrixOperator(mat['A0'])
Amu = NumpyMatrixOperator(sum(0.2 * i * mat[f'A{i}'] for i in range(1, 5)))
A = LincombOperator((A0, Amu), (1, ProjectionParameterFunctional('mu', index=0)))
B = NumpyMatrixOperator(mat['B'])
C = NumpyMatrixOperator(mat['C'])
E = NumpyMatrixOperator(mat['E'])

In [ ]:
fom = LTIModel(A, B, C, E=E)

In [ ]:
fom

In [ ]:
print(fom)

# Parametric magnitude plot

In [ ]:
w_list = np.logspace(-4, 4, 10)
mu_list = np.logspace(-6, 2, 10)
tf = np.array([[spla.norm(fom.eval_tf(w * 1j, mu=mu))
                for w in w_list]
               for mu in mu_list])

In [ ]:
fig, ax = plt.subplots()
pcm = ax.pcolormesh(w_list, mu_list, tf, shading='gouraud',
                    norm=colors.LogNorm(vmin=tf.min(), vmax=tf.max()))
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'Frequency $\omega$')
ax.set_ylabel(r'Parameter $\mu$')
ax.set_title(r'$\Vert H(i \omega, \mu) \Vert$')
_ = fig.colorbar(pcm)

# Hankel singular values

In [ ]:
mu_list_hsv = np.logspace(-6, 2, 3)
hsv_list = [fom.hsv(mu=mu) for mu in mu_list_hsv]

In [ ]:
fig, ax = plt.subplots()
for mu, hsv in zip(mu_list_hsv, hsv_list):
    ax.semilogy(hsv, '.-', label=fr'$\mu$ = {mu}')
ax.legend()
_ = ax.set_title('Hankel singular values')

# Balanced truncation

In [ ]:
from pymor.reductors.bt import BTReductor

In [ ]:
bt = BTReductor(fom, mu=1e-2)

In [ ]:
rom0 = bt.reduce(10)

In [ ]:
rom0

In [ ]:
from pymor.reductors.basic import LTIPGReductor

In [ ]:
pg = LTIPGReductor(fom, bt.W, bt.V)

In [ ]:
rom_pg = pg.reduce()

In [ ]:
rom_pg

In [ ]:
err_pg = fom - rom_pg

In [ ]:
w_list = np.logspace(-4, 4, 10)
mu_list = np.logspace(-6, 2, 10)
tf_err_pg = np.array([[spla.norm(err_pg.eval_tf(w * 1j, mu=mu))
                    for w in w_list]
                   for mu in mu_list])

In [ ]:
fig, ax = plt.subplots()
pcm = ax.pcolormesh(w_list, mu_list, tf_err_pg, shading='gouraud',
                    norm=colors.LogNorm(vmin=tf_err_pg.min(), vmax=tf_err_pg.max()))
ax.set_xscale('log')
ax.set_yscale('log')
_ = fig.colorbar(pcm)

In [ ]:
fig, ax = plt.subplots()
for mu in [1e-6, 1e-2, 1e2]:
    poles = rom_pg.poles(mu)
    ax.plot(poles.real, poles.imag, '.', label=fr'$\mu$ = {mu}')
ax.set_xscale('symlog')
ax.set_yscale('symlog')
ax.set_title('Poles of the ROM')
_ = ax.legend()

In [ ]:
fig, ax = plt.subplots()
mu_list_poles = np.logspace(-6, 2, 200)
spectral_abscisa = np.array([rom_pg.poles(mu).real.max() for mu in mu_list_poles])
ax.semilogx(mu_list_poles, spectral_abscisa)
ax.set_yscale('symlog')
ax.set_xlabel('Parameter')
_ = ax.set_title('Spectral abscisa (maximum real part of the spectrum)')

# Merging local bases

In [ ]:
mu_list_bt = [1e-6, 1e-2, 1e2]
V = fom.A.source.empty()
W = fom.A.source.empty()
for mu in mu_list_bt:
    bt_mu = BTReductor(fom, mu=mu)
    rom_bt_mu = bt_mu.reduce(10)
    V.append(bt_mu.V)
    W.append(bt_mu.W)

In [ ]:
from pymor.algorithms.pod import pod

In [ ]:
V_pod, V_sv = pod(V)

In [ ]:
W_pod, W_sv = pod(W)

In [ ]:
_ = plt.semilogy(V_sv, '.-')
_ = plt.semilogy(W_sv, '.-')

In [ ]:
V_global = V_pod[:20]
W_global = W_pod[:20]

In [ ]:
pg_VW = LTIPGReductor(fom, W_global, V_global)

In [ ]:
rom_pg_VW = pg_VW.reduce()

In [ ]:
rom_pg_VW

In [ ]:
err_pg_VW = fom - rom_pg_VW

In [ ]:
w_list = np.logspace(-4, 4, 10)
mu_list = np.logspace(-6, 2, 10)
tf_err_pg_VW = np.array([[spla.norm(err_pg_VW.eval_tf(w * 1j, mu=mu))
                    for w in w_list]
                   for mu in mu_list])

In [ ]:
fig, ax = plt.subplots()
pcm = ax.pcolormesh(w_list, mu_list, tf_err_pg_VW, shading='gouraud',
                    norm=colors.LogNorm(vmin=tf_err_pg_VW.min(), vmax=tf_err_pg_VW.max()))
ax.set_xscale('log')
ax.set_yscale('log')
_ = fig.colorbar(pcm)

In [ ]:
fig, ax = plt.subplots()
for mu in [1e-6, 1e-2, 1e2]:
    poles = rom_pg_VW.poles(mu)
    ax.plot(poles.real, poles.imag, '.', label=fr'$\mu$ = {mu}')
ax.set_xscale('symlog')
ax.set_yscale('symlog')
ax.set_title('Poles of the ROM')
_ = ax.legend()

In [ ]:
fig, ax = plt.subplots()
mu_list_poles = np.logspace(-6, 2, 200)
spectral_abscisa = np.array([rom_pg_VW.poles(mu).real.max() for mu in mu_list_poles])
ax.semilogx(mu_list_poles, spectral_abscisa)
ax.set_yscale('symlog')
ax.set_xlabel('Parameter')
_ = ax.set_title('Spectral abscisa (maximum real part of the spectrum)')

# Galerkin projection

In [ ]:
from pymor.vectorarrays.constructions import cat_arrays

In [ ]:
V_galerkin = cat_arrays([V, W])

In [ ]:
V_galerkin_pod, V_galerkin_sv = pod(V_galerkin)

In [ ]:
_ = plt.semilogy(V_galerkin_sv, '.-')

In [ ]:
Vg = V_galerkin_pod[:50]

In [ ]:
pg_Vg = LTIPGReductor(fom, Vg, Vg)

In [ ]:
rom_pg_Vg = pg_Vg.reduce()

In [ ]:
rom_pg_Vg

In [ ]:
err_pg_Vg = fom - rom_pg_Vg

In [ ]:
w_list = np.logspace(-4, 4, 10)
mu_list = np.logspace(-6, 2, 10)
tf_err_pg_Vg = np.array([[spla.norm(err_pg_Vg.eval_tf(w * 1j, mu=mu))
                    for w in w_list]
                   for mu in mu_list])

In [ ]:
fig, ax = plt.subplots()
pcm = ax.pcolormesh(w_list, mu_list, tf_err_pg_Vg, shading='gouraud',
                    norm=colors.LogNorm(vmin=tf_err_pg_Vg.min(), vmax=tf_err_pg_Vg.max()))
ax.set_xscale('log')
ax.set_yscale('log')
_ = fig.colorbar(pcm)

In [ ]:
fig, ax = plt.subplots()
for mu in [1e-6, 1e-2, 1e2]:
    poles = rom_pg_Vg.poles(mu)
    ax.plot(poles.real, poles.imag, '.', label=fr'$\mu$ = {mu}')
ax.set_xscale('symlog')
ax.set_yscale('symlog')
ax.set_title('Poles of the ROM')
_ = ax.legend()

In [ ]:
fig, ax = plt.subplots()
mu_list_poles = np.logspace(-6, 2, 200)
spectral_abscisa = np.array([rom_pg_Vg.poles(mu).real.max() for mu in mu_list_poles])
ax.semilogx(mu_list_poles, spectral_abscisa)
ax.set_yscale('symlog')
ax.set_xlabel('Parameter')
_ = ax.set_title('Spectral abscisa (maximum real part of the spectrum)')